<a href="https://colab.research.google.com/github/taras-musakovskyi/colab-jupyter-fish-models/blob/main/Species_Name_Conditioned_Single_VAE_for_Multi_Scale_Fish_Dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#!/usr/bin/env python3
"""
Size-Conditioned Single VAE for Multi-Scale Fish Dataset
Dataset: 9,498 fish crops across 3 size categories (small/medium/wide)
Training: Google Colab A100 40GB
Purpose: Part 5 - Fish inpainting into apartment scenes
"""

# ============================================================================
# CELL 1: Environment Setup & Installation
# ============================================================================
!pip install -q torch torchvision --upgrade
!pip install -q pillow numpy tqdm matplotlib

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image
import os
import zipfile
from pathlib import Path
from datetime import datetime
from tqdm.auto import tqdm
import numpy as np
from dataclasses import dataclass
from typing import Optional
import time
import gc
import matplotlib.pyplot as plt

print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"VRAM: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.2f} GB")

PyTorch version: 2.8.0+cu126
CUDA available: True
GPU: NVIDIA A100-SXM4-40GB
VRAM: 39.56 GB


In [ ]:
# ============================================================================
# CELL 2: Google Drive Authentication (for uploading results)
# ============================================================================
from google.colab import auth
from googleapiclient.discovery import build
from googleapiclient.http import MediaIoBaseUpload
import io

print("🔐 Authenticating with Google Drive for results upload...")

auth.authenticate_user()
from google.auth import default
creds, _ = default()

drive_service = build('drive', 'v3', credentials=creds)

print("✓ Authentication successful!")

# Set your results upload folder ID
GDRIVE_RESULTS_FOLDER_ID = "1aIbv7Jgad_BAcJpwTQp9sYn4_2oZToXp"# VAE - for inpaint - size conditioned

if "YOUR_" in GDRIVE_RESULTS_FOLDER_ID or "_HERE" in GDRIVE_RESULTS_FOLDER_ID:
    print("\n⚠️  WARNING: Set GDRIVE_RESULTS_FOLDER_ID to upload results!")
else:
    print(f"✓ Results will be uploaded to folder ID: {GDRIVE_RESULTS_FOLDER_ID}")

🔐 Authenticating with Google Drive for results upload...
✓ Authentication successful!
✓ Results will be uploaded to folder ID: 1aIbv7Jgad_BAcJpwTQp9sYn4_2oZToXp


In [ ]:
@dataclass
class VAEConfig:
    # Model architecture
    image_size: int = 256
    latent_dim: int = 256
    hidden_dims: list = None
    species_embed_dim: int = 128

    # Training settings
    num_epochs: int = 250
    batch_size: int = 32
    learning_rate: float = 1e-4
    beta: float = 1.0

    # Data settings
    num_workers: int = 0

    # Checkpointing
    save_every_n_epochs: int = 50
    validation_every_n_epochs: int = 50
    upload_after_epoch: int = 90
    upload_interval_epochs: int = 50

    # Paths
    output_dir: str = "/content/species_vae_output"
    dataset_dir: str = "/content/fish_dataset"

    # Species categories - CHANGED
    species_categories: dict = None

    def __post_init__(self):
        if self.hidden_dims is None:
            self.hidden_dims = [32, 64, 128, 256, 512]

        if self.species_categories is None:
            self.species_categories = {
                'guppy': 0,  # Combined guppy_female + guppy_male
                'gold_molly': 1,
                'gold_fish': 2,
                'ancistrus': 3,
                'black_molly': 4,
                'dalmatian_molly': 5
            }

config = VAEConfig()

print("="*60)
print("SPECIES-CONDITIONED VAE TRAINING CONFIGURATION")
print("="*60)
print(f"Image Size: {config.image_size}×{config.image_size}")
print(f"Latent Dim: {config.latent_dim}")
print(f"Species Embedding Dim: {config.species_embed_dim}")
print(f"Epochs: {config.num_epochs}")
print(f"Batch Size: {config.batch_size}")
print(f"Learning Rate: {config.learning_rate}")
print(f"Beta (KL weight): {config.beta}")
print(f"\nSpecies Categories: {config.species_categories}")
print("="*60)

SPECIES-CONDITIONED VAE TRAINING CONFIGURATION
Image Size: 256×256
Latent Dim: 256
Species Embedding Dim: 128
Epochs: 250
Batch Size: 32
Learning Rate: 0.0001
Beta (KL weight): 1.0

Species Categories: {'guppy': 0, 'gold_molly': 1, 'gold_fish': 2, 'ancistrus': 3, 'black_molly': 4, 'dalmatian_molly': 5}


In [ ]:
# ============================================================================
# CELL 4: Upload Dataset from Local Machine
# ============================================================================
from google.colab import files

print("📦 Upload your fish dataset ZIP file")
print("   Expected structure inside ZIP:")
print("   - small/")
print("   - medium/")
print("   - wide/")
print()

uploaded = files.upload()

# Get the uploaded filename
zip_filename = list(uploaded.keys())[0]
print(f"\n✓ Uploaded: {zip_filename}")

# Extract dataset
print("📂 Extracting dataset...")
os.makedirs(config.dataset_dir, exist_ok=True)

with zipfile.ZipFile(zip_filename, 'r') as zip_ref:
    zip_ref.extractall(config.dataset_dir)

print("✓ Dataset extracted!")

# Verify folder structure
dataset_stats = {}
for size_folder in ['small', 'medium', 'wide']:  # Hardcode instead
    folder_path = os.path.join(config.dataset_dir, size_folder)
    if os.path.exists(folder_path):
        num_images = len([f for f in os.listdir(folder_path) if f.endswith(('.jpg', '.jpeg', '.png'))])
        dataset_stats[size_folder] = num_images
        print(f"  ✓ {size_folder}: {num_images} images")
    else:
        print(f"  ⚠️  {size_folder}: NOT FOUND")
        dataset_stats[size_folder] = 0

total_images = sum(dataset_stats.values())
print(f"\n📊 Total images: {total_images}")

  ✓ small: 1771 images
  ✓ medium: 4038 images
  ✓ wide: 3689 images

📊 Total images: 9498


In [ ]:
# ============================================================================
# CELL 5: Custom Dataset with Size Conditioning
# ============================================================================
class SpeciesConditionedFishDataset(Dataset):  # CHANGED name
    """
    Dataset that provides fish images with species category labels.
    All images resized to common resolution.
    """
    def __init__(self, dataset_dir, species_categories, image_size=256):
        self.image_size = image_size
        self.species_categories = species_categories

        self.transform = transforms.Compose([
            transforms.Resize(image_size, interpolation=transforms.InterpolationMode.BILINEAR),
            transforms.CenterCrop(image_size),
            transforms.ToTensor(),
        ])

        # Collect all image paths with species labels
        self.samples = []

        # Iterate through all size folders
        for size_folder in ['small', 'medium', 'wide']:
            folder_path = os.path.join(dataset_dir, size_folder)

            if not os.path.exists(folder_path):
                print(f"⚠️  Skipping {size_folder} - folder not found")
                continue

            image_files = [
                f for f in os.listdir(folder_path)
                if f.lower().endswith(('.jpg', '.jpeg', '.png'))
            ]

            for img_file in image_files:
                # Extract species from filename
                species_name = self._extract_species(img_file)

                if species_name in species_categories:
                    species_id = species_categories[species_name]
                    img_path = os.path.join(folder_path, img_file)
                    self.samples.append((img_path, species_id))

        # Print statistics
        species_counts = {}
        for _, species_id in self.samples:
            species_name = [k for k, v in species_categories.items() if v == species_id][0]
            species_counts[species_name] = species_counts.get(species_name, 0) + 1

        print(f"\n✓ Dataset ready: {len(self.samples)} total samples")
        print(f"\nSpecies distribution:")
        for species_name, count in sorted(species_counts.items()):
            print(f"  {species_name}: {count} images")

    def _extract_species(self, filename):
        """
        Extract species name from filename.
        Format: {species}_{timestamp}_{number}_conf{confidence}.jpg
        Combines guppy_female and guppy_male into 'guppy'
        """
        # Remove extension
        name_without_ext = filename.rsplit('.', 1)[0]

        # Split by underscore
        parts = name_without_ext.split('_')

        # Find where timestamp starts (8-digit number)
        species_parts = []
        for part in parts:
            if part.isdigit() and len(part) == 8:
                break
            species_parts.append(part)

        if not species_parts:
            return 'unknown'

        # Join species parts
        species_name = '_'.join(species_parts)

        # Combine guppy variants
        if species_name in ['guppy_female', 'guppy_male']:
            return 'guppy'

        return species_name

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        img_path, species_id = self.samples[idx]

        # Load and transform image
        image = Image.open(img_path).convert('RGB')
        image = self.transform(image)

        return {
            'image': image,
            'species_id': species_id
        }

# Create dataset
print("="*60)
print("CREATING SPECIES-CONDITIONED DATASET")
print("="*60)

dataset = SpeciesConditionedFishDataset(
    config.dataset_dir,
    config.species_categories,
    config.image_size
)

# Create dataloader
train_dataloader = DataLoader(
    dataset,
    batch_size=config.batch_size,
    shuffle=True,
    num_workers=config.num_workers,
    pin_memory=True
)

print(f"\n✓ DataLoader created: {len(train_dataloader)} batches per epoch")

CREATING SPECIES-CONDITIONED DATASET

✓ Dataset ready: 9498 total samples

Species distribution:
  ancistrus: 1664 images
  black_molly: 877 images
  dalmatian_molly: 500 images
  gold_fish: 1938 images
  gold_molly: 2413 images
  guppy: 2106 images

✓ DataLoader created: 297 batches per epoch


In [ ]:
# ============================================================================
# CELL 6: Size-Conditioned VAE Architecture
# ============================================================================
class SpeciesConditionedVAE(nn.Module):  # CHANGED name
    """
    VAE with species category conditioning.  # CHANGED
    Handles all fish species in single unified latent space.  # CHANGED
    """
    def __init__(self, config):
        super().__init__()

        self.latent_dim = config.latent_dim
        self.species_embed_dim = config.species_embed_dim  # CHANGED

        # Species embedding layer - CHANGED
        num_species_categories = len(config.species_categories)
        self.species_embedding = nn.Embedding(num_species_categories, config.species_embed_dim)

        # [Rest of __init__ same, just rename size_embed_dim → species_embed_dim in comments]

        # Encoder (same)
        modules = []
        in_channels = 3
        for h_dim in config.hidden_dims:
            modules.append(
                nn.Sequential(
                    nn.Conv2d(in_channels, h_dim, kernel_size=3, stride=2, padding=1),
                    nn.BatchNorm2d(h_dim),
                    nn.LeakyReLU()
                )
            )
            in_channels = h_dim

        self.encoder = nn.Sequential(*modules)

        self.flatten_size = config.hidden_dims[-1] * (config.image_size // (2 ** len(config.hidden_dims))) ** 2

        # Latent layers (conditioned on species embedding) - CHANGED comment
        self.fc_mu = nn.Linear(self.flatten_size + config.species_embed_dim, config.latent_dim)
        self.fc_var = nn.Linear(self.flatten_size + config.species_embed_dim, config.latent_dim)

        # Decoder input (latent + species embedding) - CHANGED comment
        self.decoder_input = nn.Linear(config.latent_dim + config.species_embed_dim, self.flatten_size)

        # Decoder (same)
        modules = []
        hidden_dims_reversed = config.hidden_dims[::-1]

        for i in range(len(hidden_dims_reversed) - 1):
            modules.append(
                nn.Sequential(
                    nn.ConvTranspose2d(
                        hidden_dims_reversed[i],
                        hidden_dims_reversed[i + 1],
                        kernel_size=3,
                        stride=2,
                        padding=1,
                        output_padding=1
                    ),
                    nn.BatchNorm2d(hidden_dims_reversed[i + 1]),
                    nn.LeakyReLU()
                )
            )

        self.decoder = nn.Sequential(*modules)

        # Final layer (same)
        self.final_layer = nn.Sequential(
            nn.ConvTranspose2d(
                hidden_dims_reversed[-1],
                hidden_dims_reversed[-1],
                kernel_size=3,
                stride=2,
                padding=1,
                output_padding=1
            ),
            nn.BatchNorm2d(hidden_dims_reversed[-1]),
            nn.LeakyReLU(),
            nn.Conv2d(hidden_dims_reversed[-1], 3, kernel_size=3, padding=1),
            nn.Sigmoid()
        )

    def encode(self, x, species_id):  # CHANGED param name
        """
        Encode input image conditioned on species category.  # CHANGED
        """
        # Get species embedding - CHANGED
        species_embed = self.species_embedding(species_id)

        # Encode image
        encoded = self.encoder(x)
        encoded = torch.flatten(encoded, start_dim=1)

        # Concatenate with species embedding - CHANGED
        encoded = torch.cat([encoded, species_embed], dim=1)

        # Get mean and log variance
        mu = self.fc_mu(encoded)
        log_var = self.fc_var(encoded)

        return mu, log_var

    def reparameterize(self, mu, log_var):
        """Reparameterization trick."""
        std = torch.exp(0.5 * log_var)
        eps = torch.randn_like(std)
        return mu + eps * std

    def decode(self, z, species_id):  # CHANGED param name
        """
        Decode latent vector conditioned on species category.  # CHANGED
        """
        # Get species embedding - CHANGED
        species_embed = self.species_embedding(species_id)

        # Concatenate latent with species embedding - CHANGED
        z = torch.cat([z, species_embed], dim=1)

        # Decode
        result = self.decoder_input(z)
        result = result.view(-1, config.hidden_dims[-1],
                           config.image_size // (2 ** len(config.hidden_dims)),
                           config.image_size // (2 ** len(config.hidden_dims)))
        result = self.decoder(result)
        result = self.final_layer(result)

        return result

    def forward(self, x, species_id):  # CHANGED param name
        """Forward pass through entire VAE."""
        mu, log_var = self.encode(x, species_id)
        z = self.reparameterize(mu, log_var)
        reconstruction = self.decode(z, species_id)

        return reconstruction, mu, log_var

# Create model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = SpeciesConditionedVAE(config).to(device)  # CHANGED

print("="*60)
print("SPECIES-CONDITIONED VAE ARCHITECTURE")  # CHANGED
print("="*60)
print(model)

total_params = sum(p.numel() for p in model.parameters())
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f"\nTotal parameters: {total_params:,}")
print(f"Trainable parameters: {trainable_params:,}")
print("="*60)

SPECIES-CONDITIONED VAE ARCHITECTURE
SpeciesConditionedVAE(
  (species_embedding): Embedding(6, 128)
  (encoder): Sequential(
    (0): Sequential(
      (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): LeakyReLU(negative_slope=0.01)
    )
    (1): Sequential(
      (0): Conv2d(32, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
      (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): LeakyReLU(negative_slope=0.01)
    )
    (2): Sequential(
      (0): Conv2d(64, 128, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
      (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): LeakyReLU(negative_slope=0.01)
    )
    (3): Sequential(
      (0): Conv2d(128, 256, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
      (1): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=Tr

In [ ]:
# ============================================================================
# CELL 7: Loss Function and Optimizer
# ============================================================================
def vae_loss(recon_x, x, mu, log_var, beta=1.0):
    """
    VAE loss = Reconstruction loss + KL divergence
    """
    # Reconstruction loss (MSE)
    recon_loss = F.mse_loss(recon_x, x, reduction='sum')

    # KL divergence
    kl_loss = -0.5 * torch.sum(1 + log_var - mu.pow(2) - log_var.exp())

    # Total loss
    total_loss = recon_loss + beta * kl_loss

    return total_loss, recon_loss, kl_loss

# Optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=config.learning_rate)

# Learning rate scheduler
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
    optimizer, mode='min', factor=0.5, patience=5
)

print("✓ Loss function and optimizer configured")
print(f"  Optimizer: Adam (lr={config.learning_rate})")
print(f"  Scheduler: ReduceLROnPlateau")
print(f"  Beta (KL weight): {config.beta}")

✓ Loss function and optimizer configured
  Optimizer: Adam (lr=0.0001)
  Scheduler: ReduceLROnPlateau
  Beta (KL weight): 1.0


In [ ]:
# ============================================================================
# CELL 8: Create Output Directories
# ============================================================================
os.makedirs(config.output_dir, exist_ok=True)
os.makedirs(f"{config.output_dir}/checkpoints", exist_ok=True)
os.makedirs(f"{config.output_dir}/samples", exist_ok=True)
os.makedirs(f"{config.output_dir}/reconstructions", exist_ok=True)

print(f"✓ Output directories created at {config.output_dir}")

✓ Output directories created at /content/species_vae_output


In [ ]:
# ============================================================================
# CELL 9: Training Loop with Periodic Upload
# ============================================================================
def upload_model_to_gdrive(epoch, model_path, max_retries=3):
    """Upload model weights to Google Drive"""
    for attempt in range(max_retries):
        try:
            print(f"\n📤 Uploading model to Google Drive (attempt {attempt+1}/{max_retries})...")

            if not os.path.exists(model_path):
                print("⚠️ Model not found, skipping upload")
                return False

            timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
            remote_filename = f'size_vae_epoch_{epoch}_{timestamp}.pth'

            file_metadata = {
                'name': remote_filename,
                'parents': [GDRIVE_RESULTS_FOLDER_ID]
            }

            print(f"   Uploading: {os.path.basename(model_path)}")
            with open(model_path, 'rb') as f:
                media = MediaIoBaseUpload(
                    io.BytesIO(f.read()),
                    mimetype='application/octet-stream',
                    resumable=True
                )

            drive_service.files().create(
                body=file_metadata,
                media_body=media,
                fields='id'
            ).execute()

            print(f"✅ Model uploaded: {remote_filename}")
            return True

        except Exception as e:
            print(f"⚠️ Upload attempt {attempt+1} failed: {e}")
            if attempt < max_retries - 1:
                wait_time = 2 ** attempt
                print(f"   Retrying in {wait_time} seconds...")
                time.sleep(wait_time)
            else:
                print(f"❌ Upload failed after {max_retries} attempts")
                return False

def save_reconstructions(model, dataloader, epoch, device, num_samples=8):
    """Generate and save reconstruction examples"""
    model.eval()

    with torch.no_grad():
        # Get a batch
        batch = next(iter(dataloader))
        images = batch['image'][:num_samples].to(device)
        species_ids = batch['species_id'][:num_samples].to(device)

        # Generate reconstructions
        #recons, _, _ = model(images, size_ids)
        recons, mu, log_var = model(images, species_ids)

        # Create comparison grid
        fig, axes = plt.subplots(2, num_samples, figsize=(num_samples*2, 4))

        for i in range(num_samples):
            # Original
            axes[0, i].imshow(images[i].cpu().permute(1, 2, 0))
            axes[0, i].axis('off')
            if i == 0:
                axes[0, i].set_title('Original', fontsize=10)

            # Reconstruction
            axes[1, i].imshow(recons[i].cpu().permute(1, 2, 0))
            axes[1, i].axis('off')
            if i == 0:
                axes[1, i].set_title('Reconstructed', fontsize=10)

        plt.tight_layout()
        save_path = f"{config.output_dir}/reconstructions/epoch_{epoch:03d}.png"
        plt.savefig(save_path, dpi=100, bbox_inches='tight')
        plt.close()

        print(f"✓ Saved reconstructions: {save_path}")

    model.train()

# Training state
best_loss = float('inf')
best_model_path = f"{config.output_dir}/checkpoints/vae_best.pth"
last_uploaded_epoch = 0

print("="*60)
print("STARTING TRAINING")
print("="*60)

start_epoch = 0
checkpoint_path = f"{config.output_dir}/checkpoints/vae_best.pth"

if os.path.exists(checkpoint_path):
    checkpoint = torch.load(checkpoint_path, weights_only=False)
    model.load_state_dict(checkpoint['model_state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    start_epoch = checkpoint['epoch']
    print(f"✓ Resuming from epoch {start_epoch}")


for epoch in range(start_epoch, config.num_epochs):
    model.train()
    epoch_loss = 0.0
    epoch_recon_loss = 0.0
    epoch_kl_loss = 0.0

    progress_bar = tqdm(train_dataloader, desc=f"Epoch {epoch+1}/{config.num_epochs}")

    for batch in progress_bar:
        images = batch['image'].to(device)
        species_ids = batch['species_id'].to(device)

        # Forward pass
        recon_images, mu, log_var = model(images, species_ids)


        # Calculate loss
        loss, recon_loss, kl_loss = vae_loss(
            recon_images, images, mu, log_var, config.beta
        )

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        optimizer.step()

        # Logging
        epoch_loss += loss.item()
        epoch_recon_loss += recon_loss.item()
        epoch_kl_loss += kl_loss.item()

        progress_bar.set_postfix({
            "loss": f"{loss.item():.2f}",
            "recon": f"{recon_loss.item():.2f}",
            "kl": f"{kl_loss.item():.2f}"
        })

    # End of epoch
    avg_loss = epoch_loss / len(train_dataloader.dataset)
    avg_recon = epoch_recon_loss / len(train_dataloader.dataset)
    avg_kl = epoch_kl_loss / len(train_dataloader.dataset)

    print(f"\nEpoch {epoch+1}/{config.num_epochs}")
    print(f"  Average Loss: {avg_loss:.4f}")
    print(f"  Recon Loss: {avg_recon:.4f}")
    print(f"  KL Loss: {avg_kl:.4f}")

    # Update learning rate
    scheduler.step(avg_loss)

    # Save best model
    if avg_loss < best_loss:
        best_loss = avg_loss
        torch.save({
            'epoch': epoch + 1,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'loss': avg_loss,
            'config': config
        }, best_model_path)
        print(f"  🌟 New best model saved! Loss: {avg_loss:.4f}")

    # Upload to Google Drive
    if (epoch + 1) >= config.upload_after_epoch and \
        (epoch + 1 - last_uploaded_epoch) >= config.upload_interval_epochs:
        if upload_model_to_gdrive(epoch + 1, best_model_path):
            last_uploaded_epoch = epoch + 1

    # Save checkpoint periodically
    if (epoch + 1) % config.save_every_n_epochs == 0:
        checkpoint_path = f"{config.output_dir}/checkpoints/vae_epoch_{epoch+1}.pth"
        torch.save({
            'epoch': epoch + 1,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'loss': avg_loss,
            'config': config
        }, checkpoint_path)
        print(f"  ✓ Checkpoint saved: epoch {epoch+1}")

    # Generate reconstruction samples
    if (epoch + 1) % config.validation_every_n_epochs == 0:
        save_reconstructions(model, train_dataloader, epoch + 1, device)

print("\n" + "="*60)
print("TRAINING COMPLETE!")
print("="*60)
print(f"Best loss: {best_loss:.4f}")
print(f"Best model saved at: {best_model_path}")

# Final upload
print("\n📤 Uploading final best model...")
upload_model_to_gdrive(config.num_epochs, best_model_path)

print("\n✅ All done! Check your Google Drive for uploaded models.")

STARTING TRAINING
✓ Resuming from epoch 50


Epoch 51/250:   0%|          | 0/297 [00:00<?, ?it/s]


Epoch 51/250
  Average Loss: 1417.6947
  Recon Loss: 1014.8883
  KL Loss: 402.8063
  🌟 New best model saved! Loss: 1417.6947


Epoch 52/250:   0%|          | 0/297 [00:00<?, ?it/s]


Epoch 52/250
  Average Loss: 1408.4788
  Recon Loss: 1006.3965
  KL Loss: 402.0823
  🌟 New best model saved! Loss: 1408.4788


...


Epoch 248/250
  Average Loss: 1022.8848
  Recon Loss: 627.9399
  KL Loss: 394.9449
  🌟 New best model saved! Loss: 1022.8848


Epoch 249/250:   0%|          | 0/297 [00:00<?, ?it/s]


Epoch 249/250
  Average Loss: 1021.3020
  Recon Loss: 626.6019
  KL Loss: 394.7001
  🌟 New best model saved! Loss: 1021.3020


Epoch 250/250:   0%|          | 0/297 [00:00<?, ?it/s]


Epoch 250/250
  Average Loss: 1021.7989
  Recon Loss: 627.0103
  KL Loss: 394.7886
  ✓ Checkpoint saved: epoch 250
✓ Saved reconstructions: /content/species_vae_output/reconstructions/epoch_250.png

TRAINING COMPLETE!
Best loss: 1021.3020
Best model saved at: /content/species_vae_output/checkpoints/vae_best.pth

📤 Uploading final best model...

📤 Uploading model to Google Drive (attempt 1/3)...
   Uploading: vae_best.pth
✅ Model uploaded: size_vae_epoch_250_20251010_164503.pth

✅ All done! Check your Google Drive for uploaded models.
